In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 91.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import zipfile
import os

def extract_with_progress(zip_path, extract_path):
    # Create extraction directory if it doesn't exist
    os.makedirs(extract_path, exist_ok=True)

    # Get total number of files in the zip
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        total_files = len(zip_ref.infolist())

    print(f"Extracting {total_files} files to: {extract_path}")

    # Extract with progress tracking and proper path handling
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        for i, file in enumerate(zip_ref.infolist(), 1):
            # Extract each file to the specified directory
            zip_ref.extract(file.filename, extract_path)

            # Calculate and display progress
            progress = (i / total_files) * 100
            print(f"\rProgress: {progress:.1f}% ({i}/{total_files})", end='', flush=True)

    print("\nExtraction complete!")
    print(f"All files successfully extracted to: {extract_path}")

# Usage example
zip_path = "/content/drive/MyDrive/AFM dataset.zip"  # Colab path
extract_path = "/content/datasets/AFM dataset"  # Where to extract

extract_with_progress(zip_path, extract_path)

Extracting 82649 files to: /content/datasets/AFM dataset
Progress: 100.0% (82649/82649)
Extraction complete!
All files successfully extracted to: /content/datasets/AFM dataset


In [ ]:
import os
import cv2
from PIL import Image
import shutil
from tqdm import tqdm

# Convert bounding box to YOLO format
def convert_to_yolo_format(box, img_width, img_height):
    x_center = (box[0] + box[2]) / 2.0 / img_width
    y_center = (box[1] + box[3]) / 2.0 / img_height
    width = (box[2] - box[0]) / img_width
    height = (box[3] - box[1]) / img_height
    return x_center, y_center, width, height

# Emotion classes mapping
class_names = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
class_to_id = {name: idx for idx, name in enumerate(class_names)}

# Paths
source_base = "/content/datasets/AFM dataset/AFM dataset"
output_base = "/content/AFM_YOLO"

# Load OpenCV Haar Cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

print("Starting YOLO format conversion...\n")

# Process both train and test
for split in ["train", "test"]:
    print(f"Processing split: {split}")

    src_dir = os.path.join(source_base, split)
    img_out_dir = os.path.join(output_base, "images", split)
    lbl_out_dir = os.path.join(output_base, "labels", split)
    os.makedirs(img_out_dir, exist_ok=True)
    os.makedirs(lbl_out_dir, exist_ok=True)

    # List classes in dataset
    class_dirs = [d for d in os.listdir(src_dir) if os.path.isdir(os.path.join(src_dir, d))]

    for class_name in class_dirs:
        class_id = class_to_id[class_name]
        class_dir = os.path.join(src_dir, class_name)
        image_files = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

        print(f"  - Processing class: {class_name} ({len(image_files)} images)")

        for filename in tqdm(image_files, desc=f"    -> {class_name}"):
            img_path = os.path.join(class_dir, filename)
            img = cv2.imread(img_path)
            if img is None:
                continue

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = face_cascade.detectMultiScale(gray, 1.1, 4)

            if len(faces) == 0:
                continue  # Skip images without detected faces

            h, w = img.shape[:2]

            # Save the image
            out_img_path = os.path.join(img_out_dir, filename)
            shutil.copy(img_path, out_img_path)

            # Save label file
            label_path = os.path.join(lbl_out_dir, os.path.splitext(filename)[0] + ".txt")
            with open(label_path, "w") as f:
                for (x, y, bw, bh) in faces:
                    box = (x, y, x + bw, y + bh)
                    x_c, y_c, bw_n, bh_n = convert_to_yolo_format(box, w, h)
                    f.write(f"{class_id} {x_c:.6f} {y_c:.6f} {bw_n:.6f} {bh_n:.6f}\n")

    print(f"Finished processing split: {split}\n")

print("✅ Dataset successfully converted to YOLO format!")
print(f"Images and labels are saved under: {output_base}")


Starting YOLO format conversion...

Processing split: train
  - Processing class: happy (14161 images)


    -> happy: 100%|██████████| 14161/14161 [00:56<00:00, 250.41it/s]


  - Processing class: disgust (4899 images)


    -> disgust: 100%|██████████| 4899/4899 [00:31<00:00, 157.99it/s]


  - Processing class: angry (10405 images)


...

  - Processing class: surprise (1590 images)


    -> surprise: 100%|██████████| 1590/1590 [00:05<00:00, 311.22it/s]


  - Processing class: fear (1644 images)


    -> fear: 100%|██████████| 1644/1644 [00:05<00:00, 287.13it/s]

Finished processing split: test

✅ Dataset successfully converted to YOLO format!
Images and labels are saved under: /content/AFM_YOLO


In [ ]:
# Create dataset.yaml file
dataset_yaml = """
path: /content/AFM_YOLO
train: images/train
val: images/test

nc: 7
names: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
"""

with open('/content/AFM_YOLO/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # or yolov8s.pt / yolov8m.pt / yolov8l.pt

# Train the model
model.train(data='/content/AFM_YOLO/dataset.yaml', epochs=30, imgsz=416, batch=64)

Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/AFM_YOLO/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

train: Scanning /content/AFM_YOLO/labels/train.cache... 32589 images, 0 backgrounds, 0 corrupt: 100%|██████████| 32589/32589 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 1.8±4.0 ms, read: 129.1±116.5 MB/s, size: 4.3 KB)


val: Scanning /content/AFM_YOLO/labels/test.cache... 4760 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4760/4760 [00:00<?, ?it/s]


Plotting labels to runs/detect/train3/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train3
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      4/100      9.16G     0.7728      1.453      1.292         40        640: 100%|██████████| 510/510 [08:47<00:00,  1.03s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 38/38 [00:38<00:00,  1.00s/it]


                   all       4760       4772        0.3      0.674      0.366      0.295

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      5/100      9.15G      0.735      1.403      1.267        179        640:  20%|██        | 103/510 [01:45<06:57,  1.03s/it]


KeyboardInterrupt: 

In [ ]:
!zip -r yolov8_checkpoint.zip runs/detect/train3

  adding: runs/detect/train3/ (stored 0%)
  adding: runs/detect/train3/args.yaml (deflated 52%)
  adding: runs/detect/train3/results.csv (deflated 52%)
  adding: runs/detect/train3/train_batch1.jpg (deflated 8%)
  adding: runs/detect/train3/weights/ (stored 0%)
  adding: runs/detect/train3/weights/best.pt (deflated 9%)
  adding: runs/detect/train3/weights/last.pt (deflated 9%)
  adding: runs/detect/train3/labels.jpg (deflated 52%)
  adding: runs/detect/train3/labels_correlogram.jpg (deflated 45%)
  adding: runs/detect/train3/train_batch2.jpg (deflated 5%)
  adding: runs/detect/train3/train_batch0.jpg (deflated 6%)


In [ ]:
!cp -r /content/yolov8_checkpoint.zip /content/drive/MyDrive/yolo_train_backup/

resume training

In [ ]:
!unzip /content/drive/MyDrive/yolo_train_backup/yolov8_checkpoint.zip -d /content/train

Archive:  /content/drive/MyDrive/yolo_train_backup/yolov8_checkpoint.zip
   creating: /content/train/runs/detect/train3/
  inflating: /content/train/runs/detect/train3/args.yaml  
  inflating: /content/train/runs/detect/train3/results.csv  
  inflating: /content/train/runs/detect/train3/train_batch1.jpg  
   creating: /content/train/runs/detect/train3/weights/
  inflating: /content/train/runs/detect/train3/weights/best.pt  
  inflating: /content/train/runs/detect/train3/weights/last.pt  
  inflating: /content/train/runs/detect/train3/labels.jpg  
  inflating: /content/train/runs/detect/train3/labels_correlogram.jpg  
  inflating: /content/train/runs/detect/train3/train_batch2.jpg  
  inflating: /content/train/runs/detect/train3/train_batch0.jpg  


In [ ]:
!yolo task=detect \
  mode=train \
  model=/content/train/runs/detect/train3/weights/last.pt \
  data=/content/AFM_YOLO/dataset.yaml \
  epochs=30 \
  imgsz=416 \
  batch=64 \
  verbose=False \
  plots=False

Ultralytics 8.3.128 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/AFM_YOLO/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=416, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/train/runs/detect/train3/weights/last.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspect

In [ ]:
!zip -r yolov8_checkpoint2.zip runs/detect/train3

  adding: runs/detect/train3/ (stored 0%)
  adding: runs/detect/train3/weights/ (stored 0%)
  adding: runs/detect/train3/weights/last.pt (deflated 9%)
  adding: runs/detect/train3/weights/best.pt (deflated 9%)
  adding: runs/detect/train3/results.csv (deflated 60%)
  adding: runs/detect/train3/args.yaml (deflated 52%)


In [ ]:
!cp -r /content/yolov8_checkpoint2.zip /content/drive/MyDrive/yolo_train_backup/

In [ ]:
from google.colab import files
files.download('runs/detect/train3/weights/best.pt')
files.download('runs/detect/train3/weights/last.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>